# Tâche 3 : Détection de Fraude dans les Documents d'Identité

Ce notebook a pour objectif de résoudre la troisième tâche du challenge ANIP : détecter les fraudes et falsifications dans les documents d'identité officiels.

## Objectif

Développer une IA capable de distinguer automatiquement les documents authentiques des documents falsifiés avec la meilleure précision possible.

## Types de Documents Analysés

Le dataset contient plusieurs types de documents d'identité :
- **Arizona DL** : Permis de conduire d'Arizona (États-Unis)
- **ESP** : Documents espagnols
- **EST** : Documents estoniens
- **RUS** : Documents russes

## Classes de Falsification

Pour chaque type de document, nous devons classifier :
- **Normal** : Documents authentiques
- **Forgery 1-4** : Différents types de falsifications

## Stratégie Adoptée

Notre approche sera systématique et basée sur les meilleures pratiques de détection de fraude :

1. **Analyse Exploratoire des Données** : Comprendre la distribution des classes, types de documents, et qualité des images
2. **Préparation des Données** : Organisation par type de document et classe, normalisation, augmentation
3. **Extraction de Caractéristiques** : Techniques de vision par ordinateur et analyse d'images
4. **Modélisation** : Classification multi-classe avec Deep Learning et techniques spécialisées
5. **Évaluation** : Métriques de précision, rappel, F1-score par classe et type de document
6. **Prédiction Finale** : Application sur les données de test et génération de la soumission

In [ ]:
# Importation des bibliothèques fondamentales
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import cv2
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (12, 8)
print("Bibliothèques de base importées.")

## 1. Configuration de l'Environnement

Définition des chemins et exploration de la structure des données pour la détection de fraude documentaire.

In [ ]:
# Configuration des chemins de données
from pathlib import Path
import os

# Configuration pour l'environnement local
DATA_ROOT = Path('/workspaces/anip-facial-ocr-challenge/dataset_tache_3/dataset_tache_3')
TRAIN_DIR = DATA_ROOT / 'train'
TEST_DIR = DATA_ROOT / 'test'  # Données de test (si disponibles)
SUBMISSIONS_DIR = Path('/workspaces/anip-facial-ocr-challenge/submissions')

print("Configuration des chemins:")
print(f"   Dataset root: {DATA_ROOT}")
print(f"   Train directory: {TRAIN_DIR}")
print(f"   Test directory: {TEST_DIR}")
print(f"   Submissions: {SUBMISSIONS_DIR}")

# Vérification des chemins
train_exists = TRAIN_DIR.exists()
test_exists = TEST_DIR.exists()

print(f"\nVérification:")
print(f"   Train folder: {'OK' if train_exists else 'MANQUANT'} {TRAIN_DIR}")
print(f"   Test folder: {'OK' if test_exists else 'MANQUANT'} {TEST_DIR}")

# Exploration de la structure des données d'entraînement
if train_exists:
    document_types = [d.name for d in TRAIN_DIR.iterdir() if d.is_dir()]
    print(f"\nTypes de documents trouvés: {document_types}")
    
    # Pour chaque type de document, explorer les classes
    for doc_type in document_types:
        doc_path = TRAIN_DIR / doc_type
        classes = [c.name for c in doc_path.iterdir() if c.is_dir()]
        print(f"   {doc_type}: {classes}")
        
        # Compter les images dans chaque classe
        for class_name in classes:
            class_path = doc_path / class_name
            image_files = list(class_path.glob('*.png')) + list(class_path.glob('*.jpg')) + list(class_path.glob('*.jpeg'))
            print(f"      {class_name}: {len(image_files)} images")
else:
    print("   ATTENTION: Les données d'entraînement ne sont pas disponibles")
    document_types = []

# Exploration des données de test (si disponibles)
if test_exists:
    print(f"\nExploration des données de test:")
    test_structure = [item.name for item in TEST_DIR.iterdir()]
    print(f"   Structure: {test_structure}")
else:
    print("\n   Les données de test ne sont pas encore disponibles localement")

## 2. Analyse Exploratoire des Données (AED)

### 2.1 Collecte et Structuration des Métadonnées

Nous allons analyser la distribution des documents par type et catégorie (authentique vs falsifié).

In [ ]:
# Construction du dataset principal
print("CONSTRUCTION DU DATASET")
print("="*50)

data_list = []

if train_exists and document_types:
    for doc_type in document_types:
        doc_path = TRAIN_DIR / doc_type
        
        if not doc_path.exists():
            continue
            
        classes = [c.name for c in doc_path.iterdir() if c.is_dir()]
        
        for class_name in classes:
            class_path = doc_path / class_name
            
            # Recherche des images avec extensions robustes
            image_files = []
            for ext in ['*.png', '*.jpg', '*.jpeg', '*.PNG', '*.JPG', '*.JPEG']:
                image_files.extend(list(class_path.glob(ext)))
            
            for img_path in image_files:
                data_list.append({
                    'file_path': str(img_path),
                    'filename': img_path.name,
                    'document_type': doc_type,
                    'class_name': class_name,
                    'is_authentic': 1 if class_name == 'normal' else 0,
                    'forgery_type': class_name if class_name != 'normal' else 'authentic'
                })

    # Créer le DataFrame principal
    df_train = pd.DataFrame(data_list)
    
    print(f"Dataset construit avec succès:")
    print(f"   Total images: {len(df_train)}")
    print(f"   Types de documents: {df_train['document_type'].nunique()}")
    print(f"   Classes: {df_train['class_name'].nunique()}")
    
    if len(df_train) > 0:
        print("\nAperçu des données:")
        print(df_train.head())
        
        print("\nDistribution par type de document:")
        print(df_train['document_type'].value_counts())
        
        print("\nDistribution par classe:")
        print(df_train['class_name'].value_counts())
        
        print("\nDistribution authentique vs falsifié:")
        print(df_train['is_authentic'].value_counts())
        
else:
    print("ATTENTION: Impossible de construire le dataset - données manquantes")
    df_train = pd.DataFrame()

# Informations sur le dataset
if len(df_train) > 0:
    print(f"\nInformations détaillées:")
    print(df_train.info())

## 3. Analyse des Fichiers Ground Truth (GT) pour l'OCR

### Objectif

Les fichiers GT contiennent les annotations OCR de référence pour chaque document. Cette analyse nous permet de :

1. **Comprendre la structure des données de sortie** attendues pour la soumission
2. **Identifier les champs OCR communs** à tous les types de documents
3. **Détecter les champs spécifiques** à chaque pays/type de document
4. **Préparer le format de prédiction** pour les données de test

### Champs OCR attendus

D'après les exemples que nous avons vus :
- **Champs d'identité** : `surname`, `given_name`, `birthday`, `gender`
- **Champs de document** : `country_code`, `issue_date`, `expire_date`, `card_num`, `personal_num`
- **Champs spécifiques** : `second_surname` (ESP), `place_of_birth` (EST), etc.

In [ ]:
import json

# Analyse des fichiers Ground Truth (GT) pour comprendre la structure OCR
print("ANALYSE DES FICHIERS GROUND TRUTH (GT)")
print("="*45)

gt_data = []
all_fields = set()
field_by_document = {}

if train_exists and document_types:
    for doc_type in document_types:
        gt_path = TRAIN_DIR / doc_type / 'gt'
        
        if gt_path.exists():
            # Rechercher les fichiers JSON
            json_files = list(gt_path.glob('*.json'))
            
            print(f"\n--- {doc_type.upper()} ---")
            print(f"Fichiers GT trouvés: {len(json_files)}")
            
            # Analyser les premiers fichiers pour comprendre la structure
            sample_files = json_files[:5]  # Analyser 5 fichiers par type
            
            for json_file in sample_files:
                try:
                    with open(json_file, 'r', encoding='utf-8') as f:
                        gt_content = json.load(f)
                    
                    # Stocker pour analyse globale
                    gt_data.append({
                        'document_type': doc_type,
                        'filename': json_file.name,
                        'gt_content': gt_content
                    })
                    
                    # Collecter tous les champs
                    for field in gt_content.keys():
                        all_fields.add(field)
                        
                        if doc_type not in field_by_document:
                            field_by_document[doc_type] = set()
                        field_by_document[doc_type].add(field)
                    
                    # Afficher un exemple
                    if json_file == sample_files[0]:  # Premier fichier seulement
                        print(f"\nExemple GT ({json_file.name}):")
                        for key, value in gt_content.items():
                            print(f"   {key}: '{value}'")
                            
                except Exception as e:
                    print(f"Erreur lecture {json_file.name}: {e}")

    print(f"\nRÉSUMÉ DE L'ANALYSE:")
    print(f"Total champs OCR identifiés: {len(all_fields)}")
    print(f"Champs: {sorted(all_fields)}")
    
    # Identifier les champs communs à tous les types de documents
    if len(field_by_document) > 1:
        common_fields = set.intersection(*field_by_document.values())
        print(f"\nChamps communs à tous les documents: {sorted(common_fields)}")
        
        # Champs spécifiques par type
        print(f"\nChamps spécifiques par type:")
        for doc_type, fields in field_by_document.items():
            specific = fields - common_fields
            if specific:
                print(f"   {doc_type}: {sorted(specific)}")
    
else:
    print("Impossible d'analyser les fichiers GT - données manquantes")

## 4. Analyse des Propriétés des Images

### Objectifs de cette analyse

1. **Dimensions et formats** : Comprendre la variabilité des tailles d'images pour le preprocessing
2. **Qualité et résolution** : Évaluer la qualité des images pour l'OCR
3. **Cohérence par type** : Vérifier si chaque type de document a des caractéristiques communes
4. **Preprocessing nécessaire** : Définir les transformations à appliquer

### Importance pour l'OCR

- **Résolution** : Une résolution insuffisante peut compromettre la lecture des textes
- **Dimensions** : Nécessaire pour définir la taille d'entrée des modèles
- **Format** : RGB vs Grayscale peut affecter les performances OCR

In [ ]:
if len(df_train) > 0:
    print("ANALYSE DES PROPRIÉTÉS DES IMAGES")
    print("="*40)
    
    # Échantillonner pour l'analyse (pour éviter de traiter toutes les images)
    sample_size = min(100, len(df_train))
    sample_df = df_train.sample(n=sample_size, random_state=42)
    
    image_properties = []
    
    print(f"Analyse d'un échantillon de {sample_size} images...")
    
    for idx, row in sample_df.iterrows():
        try:
            img_path = Path(row['file_path'])
            
            # Lire l'image
            img = Image.open(img_path)
            img_array = np.array(img)
            
            # Calculer les propriétés
            file_size = img_path.stat().st_size / (1024*1024)  # Taille en MB
            
            properties = {
                'document_type': row['document_type'],
                'class_name': row['class_name'],
                'width': img.width,
                'height': img.height,
                'channels': len(img_array.shape),
                'mode': img.mode,
                'file_size_mb': file_size,
                'aspect_ratio': img.width / img.height,
                'total_pixels': img.width * img.height
            }
            
            # Statistiques de luminosité
            if len(img_array.shape) == 3:  # Image couleur
                gray = np.mean(img_array, axis=2)
                properties['mean_brightness'] = np.mean(gray)
                properties['std_brightness'] = np.std(gray)
            else:  # Image grayscale
                properties['mean_brightness'] = np.mean(img_array)
                properties['std_brightness'] = np.std(img_array)
            
            image_properties.append(properties)
            
        except Exception as e:
            print(f"Erreur analyse {row['filename']}: {e}")
    
    # Créer DataFrame des propriétés
    props_df = pd.DataFrame(image_properties)
    
    if len(props_df) > 0:
        print(f"\nImages analysées avec succès: {len(props_df)}")
        
        # Statistiques globales
        print(f"\nSTATISTIQUES GLOBALES:")
        print(f"   Largeur - Min: {props_df['width'].min()}, Max: {props_df['width'].max()}, Moyenne: {props_df['width'].mean():.0f}")
        print(f"   Hauteur - Min: {props_df['height'].min()}, Max: {props_df['height'].max()}, Moyenne: {props_df['height'].mean():.0f}")
        print(f"   Ratio aspect - Min: {props_df['aspect_ratio'].min():.2f}, Max: {props_df['aspect_ratio'].max():.2f}")
        print(f"   Taille fichier - Min: {props_df['file_size_mb'].min():.2f}MB, Max: {props_df['file_size_mb'].max():.2f}MB")
        
        # Modes d'image
        print(f"\nMODES D'IMAGE:")
        mode_counts = props_df['mode'].value_counts()
        for mode, count in mode_counts.items():
            percentage = count / len(props_df) * 100
            print(f"   {mode}: {count} images ({percentage:.1f}%)")
        
        # Analyse par type de document
        print(f"\nANALYSE PAR TYPE DE DOCUMENT:")
        for doc_type in props_df['document_type'].unique():
            doc_props = props_df[props_df['document_type'] == doc_type]
            print(f"\n   --- {doc_type.upper()} ---")
            print(f"   Dimensions moyennes: {doc_props['width'].mean():.0f} x {doc_props['height'].mean():.0f}")
            print(f"   Luminosité moyenne: {doc_props['mean_brightness'].mean():.1f}")
            print(f"   Taille moyenne: {doc_props['file_size_mb'].mean():.2f}MB")
    
    else:
        print("Aucune image analysée avec succès")
        
else:
    print("Pas de données pour analyser les images")

## 5. Visualisation d'Exemples d'Images par Type et Classe

### Objectifs

1. **Inspection visuelle** : Comprendre les différences entre documents authentiques et falsifiés
2. **Qualité OCR** : Évaluer la lisibilité des textes dans les documents
3. **Patterns de falsification** : Identifier les caractéristiques visuelles des différents types de fraude
4. **Preprocessing** : Définir les techniques de nettoyage et amélioration d'image nécessaires

### Ce qu'il faut observer

- **Netteté du texte** : Les documents falsifiés peuvent avoir des textes flous ou pixélisés
- **Cohérence des polices** : Les falsifications peuvent mélanger différentes polices
- **Alignement** : Les informations peuvent être mal alignées dans les falsifications
- **Qualité d'impression** : Différences de qualité entre authentiques et falsifiés

In [ ]:
if len(df_train) > 0:
    print("VISUALISATION D'EXEMPLES D'IMAGES")
    print("="*40)
    
    # Sélectionner des exemples représentatifs
    examples_per_type = 2
    examples_per_class = 1
    
    fig, axes = plt.subplots(4, 6, figsize=(20, 16))  # 4 types de documents x 6 classes max
    axes = axes.flatten()
    
    plot_idx = 0
    
    for doc_type in sorted(df_train['document_type'].unique()):
        print(f"\n--- Exemples pour {doc_type.upper()} ---")
        
        doc_data = df_train[df_train['document_type'] == doc_type]
        classes = sorted(doc_data['class_name'].unique())
        
        for class_name in classes:
            class_data = doc_data[doc_data['class_name'] == class_name]
            
            if len(class_data) > 0 and plot_idx < len(axes):
                # Prendre le premier exemple de cette classe
                example = class_data.iloc[0]
                
                try:
                    img = Image.open(example['file_path'])
                    
                    # Redimensionner pour l'affichage si nécessaire
                    if img.width > 800 or img.height > 600:
                        img.thumbnail((800, 600), Image.Resampling.LANCZOS)
                    
                    axes[plot_idx].imshow(img)
                    axes[plot_idx].set_title(f"{doc_type}\n{class_name}", fontsize=10)
                    axes[plot_idx].axis('off')
                    
                    print(f"   {class_name}: {example['filename']}")
                    
                except Exception as e:
                    axes[plot_idx].text(0.5, 0.5, f"Erreur\n{doc_type}\n{class_name}", 
                                       ha='center', va='center', transform=axes[plot_idx].transAxes)
                    axes[plot_idx].axis('off')
                    print(f"   {class_name}: Erreur - {e}")
                
                plot_idx += 1
    
    # Masquer les axes inutilisés
    for i in range(plot_idx, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle('Exemples d\'Images par Type de Document et Classe', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Analyser les différences visuelles observées
    print(f"\nOBSERVATIONS POUR L'OCR:")
    print("="*30)
    print("1. Vérifier la netteté et lisibilité du texte dans chaque type")
    print("2. Observer les différences de qualité entre 'normal' et 'forgery_X'")
    print("3. Identifier les zones de texte importantes pour l'extraction OCR")
    print("4. Noter les variations de format et layout entre types de documents")
    
else:
    print("Pas de données pour la visualisation")

## 6. Préparation des Données pour l'Entraînement

### Stratégie de Modélisation

Basé sur notre analyse, nous allons implémenter une approche **hybride OCR + Classification** :

1. **Modèle OCR** : Extraire les champs textuels structurés des documents
2. **Modèle de Classification** : Détecter si le document est authentique ou falsifié
3. **Pipeline intégré** : Combiner les deux approches pour une prédiction complète

### Défis identifiés

1. **Variabilité des dimensions** : Les documents ont des tailles très différentes (600x375 à 1519x1069)
2. **Déséquilibre des classes** : 78.4% de falsifiés vs 21.6% d'authentiques
3. **Diversité des formats** : Chaque pays a un layout de document différent
4. **Qualité variable** : Certaines falsifications peuvent dégrader la qualité OCR

### Plan d'action

1. **Normalisation des images** : Redimensionnement et amélioration de qualité
2. **Split stratifié** : Maintenir la distribution par type de document et classe
3. **Augmentation des données** : Techniques spécifiques pour améliorer la robustesse OCR

In [ ]:
if len(df_train) > 0:
    print("CRÉATION DU SPLIT TRAIN/VALIDATION STRATIFIÉ")
    print("="*50)
    
    # Créer une stratification combinée document_type + class_name
    df_train['stratify_key'] = df_train['document_type'] + '_' + df_train['class_name']
    
    print("Groupes de stratification:")
    stratify_counts = df_train['stratify_key'].value_counts()
    for group, count in stratify_counts.items():
        print(f"   {group}: {count} images")
    
    # Vérifier qu'aucun groupe n'a moins de 2 échantillons (nécessaire pour le split)
    min_samples = stratify_counts.min()
    if min_samples < 2:
        print(f"\nATTENTION: Certains groupes ont moins de 2 échantillons (min: {min_samples})")
        print("Utilisation d'une stratification simplifiée par document_type seulement")
        stratify_column = df_train['document_type']
    else:
        stratify_column = df_train['stratify_key']
    
    # Split 80/20 avec stratification
    X = df_train[['file_path', 'filename', 'document_type', 'class_name', 'is_authentic', 'forgery_type']]
    y = df_train['is_authentic']  # Target principal: authentique vs falsifié
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y,
        test_size=0.2,
        random_state=42,
        stratify=stratify_column
    )
    
    print(f"\nSplit réalisé:")
    print(f"   Train: {len(X_train)} images ({len(X_train)/len(df_train)*100:.1f}%)")
    print(f"   Validation: {len(X_val)} images ({len(X_val)/len(df_train)*100:.1f}%)")
    
    # Vérifier la distribution dans chaque ensemble
    print(f"\nDistribution par type de document:")
    print("TRAIN:")
    train_doc_dist = X_train['document_type'].value_counts(normalize=True) * 100
    for doc_type, pct in train_doc_dist.items():
        print(f"   {doc_type}: {pct:.1f}%")
    
    print("VALIDATION:")
    val_doc_dist = X_val['document_type'].value_counts(normalize=True) * 100
    for doc_type, pct in val_doc_dist.items():
        print(f"   {doc_type}: {pct:.1f}%")
    
    print(f"\nDistribution authentique/falsifié:")
    print(f"TRAIN - Authentiques: {(y_train==1).sum()} ({(y_train==1).mean()*100:.1f}%)")
    print(f"TRAIN - Falsifiés: {(y_train==0).sum()} ({(y_train==0).mean()*100:.1f}%)")
    print(f"VAL - Authentiques: {(y_val==1).sum()} ({(y_val==1).mean()*100:.1f}%)")
    print(f"VAL - Falsifiés: {(y_val==0).sum()} ({(y_val==0).mean()*100:.1f}%)")
    
else:
    print("Pas de données pour créer le split")
    X_train, X_val, y_train, y_val = None, None, None, None

### 6.1 Train/Validation Split Stratifié

Nous devons créer un split intelligent qui préserve :
- La distribution des types de documents
- La distribution des classes (normal vs forgery_X) 
- La représentativité de chaque pays

In [ ]:
# Importation des bibliothèques de base pour commencer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print("Bibliothèques de base importées.")
print("Installation des packages PyTorch en cours...")

# Vérifier PyTorch
try:
    import torch
    print(f"✓ PyTorch disponible - Version: {torch.__version__}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")
except ImportError:
    print("✗ PyTorch non disponible - sera installé si nécessaire")
    device = 'cpu'

print("Configuration de base terminée.")

In [ ]:
# Installation des packages nécessaires
import subprocess
import sys

def install_package(package):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installé avec succès")
    except subprocess.CalledProcessError:
        print(f"✗ Erreur lors de l'installation de {package}")

print("Installation des packages nécessaires...")

# Packages essentiels pour PyTorch et OCR
packages = [
    "torch",
    "torchvision", 
    "scikit-learn",
    "pytesseract",
    "easyocr"
]

for package in packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✓ {package} déjà installé")
    except ImportError:
        print(f"Installation de {package}...")
        install_package(package)

print("Installation terminée.")

## Analyse de la Distribution des Classes

### Distribution par Type de Document et Classe

Cette analyse nous permet de comprendre :
1. **L'équilibre des classes** : Y a-t-il plus de documents authentiques que de falsifiés ?
2. **La représentation par type de document** : Certains types sont-ils sur ou sous-représentés ?
3. **Les types de falsification** : Quels sont les types de fraude les plus courants ?
4. **La qualité des données** : Y a-t-il des incohérences ou des problèmes ?

### Métriques Importantes

- **Ratio Authentique/Falsifié** : Important pour ajuster les métriques d'évaluation
- **Distribution par Document** : Arizona DL vs ESP vs EST vs RUS
- **Types de Falsification** : Forgery 1, 2, 3, 4 et leurs spécificités
- **Taille des Images** : Cohérence des dimensions pour le preprocessing

### Stratégie de Modélisation

Basé sur cette analyse, nous pourrons décider :
- **Approche de classification** : Binaire (authentique/falsifié) ou multi-classe
- **Techniques de balancement** : Si les classes sont déséquilibrées
- **Architecture de modèle** : CNN spécialisé pour chaque type de document ou modèle global
- **Métriques d'évaluation** : Précision, rappel, F1-score adaptés au déséquilibre

In [ ]:
if len(df_train) > 0:
    # Analyse de la distribution globale
    print("ANALYSE DE LA DISTRIBUTION DES CLASSES")
    print("="*45)
    
    # Distribution par authenticité
    auth_dist = df_train['is_authentic'].value_counts()
    auth_pct = df_train['is_authentic'].value_counts(normalize=True) * 100
    
    print("Distribution Authentique vs Falsifié:")
    print(f"   Authentiques: {auth_dist[1]:,} images ({auth_pct[1]:.1f}%)")
    print(f"   Falsifiés: {auth_dist[0]:,} images ({auth_pct[0]:.1f}%)")
    
    # Distribution par type de document
    print(f"\nDistribution par Type de Document:")
    for doc_type in df_train['document_type'].unique():
        count = (df_train['document_type'] == doc_type).sum()
        percentage = count / len(df_train) * 100
        print(f"   {doc_type}: {count:,} images ({percentage:.1f}%)")
    
    # Distribution par classe détaillée
    print(f"\nDistribution par Classe:")
    class_dist = df_train['class_name'].value_counts()
    for class_name in class_dist.index:
        count = class_dist[class_name]
        percentage = count / len(df_train) * 100
        print(f"   {class_name}: {count:,} images ({percentage:.1f}%)")
        
else:
    print("Pas de données à analyser - dataset vide")

In [ ]:
if len(df_train) > 0:
    # Visualisations de la distribution
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Distribution Authentique vs Falsifié
    auth_counts = df_train['is_authentic'].value_counts()
    labels = ['Falsifié', 'Authentique']
    colors = ['lightcoral', 'lightgreen']
    
    axes[0, 0].pie(auth_counts.values, labels=labels, autopct='%1.1f%%', colors=colors)
    axes[0, 0].set_title('Distribution Authentique vs Falsifié')
    
    # 2. Distribution par Type de Document
    doc_counts = df_train['document_type'].value_counts()
    axes[0, 1].bar(doc_counts.index, doc_counts.values, color='skyblue')
    axes[0, 1].set_title('Distribution par Type de Document')
    axes[0, 1].set_xlabel('Type de Document')
    axes[0, 1].set_ylabel('Nombre d\'Images')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Distribution par Classe
    class_counts = df_train['class_name'].value_counts()
    axes[1, 0].bar(class_counts.index, class_counts.values, color='lightsteelblue')
    axes[1, 0].set_title('Distribution par Classe')
    axes[1, 0].set_xlabel('Classe')
    axes[1, 0].set_ylabel('Nombre d\'Images')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Heatmap Type de Document vs Classe
    cross_tab = pd.crosstab(df_train['document_type'], df_train['class_name'])
    sns.heatmap(cross_tab, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1])
    axes[1, 1].set_title('Heatmap Document × Classe')
    axes[1, 1].set_xlabel('Classe')
    axes[1, 1].set_ylabel('Type de Document')
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Pas de données pour les visualisations")

In [ ]:
if len(df_train) > 0:
    # Analyse détaillée par type de document
    print("ANALYSE DÉTAILLÉE PAR TYPE DE DOCUMENT")
    print("="*45)
    
    for doc_type in df_train['document_type'].unique():
        print(f"\n--- {doc_type.upper()} ---")
        
        # Filtrer les données pour ce type de document
        doc_data = df_train[df_train['document_type'] == doc_type]
        
        print(f"Total images: {len(doc_data)}")
        
        # Distribution des classes pour ce type de document
        class_dist = doc_data['class_name'].value_counts()
        print("Distribution des classes:")
        for class_name in class_dist.index:
            count = class_dist[class_name]
            percentage = count / len(doc_data) * 100
            print(f"   {class_name}: {count} images ({percentage:.1f}%)")
        
        # Ratio authentique/falsifié
        auth_count = (doc_data['is_authentic'] == 1).sum()
        fake_count = (doc_data['is_authentic'] == 0).sum()
        
        print(f"Authentiques: {auth_count} ({auth_count/len(doc_data)*100:.1f}%)")
        print(f"Falsifiés: {fake_count} ({fake_count/len(doc_data)*100:.1f}%)")
        
        # Types de falsification
        if fake_count > 0:
            forgery_types = doc_data[doc_data['is_authentic'] == 0]['forgery_type'].value_counts()
            print("Types de falsification:")
            for forgery_type in forgery_types.index:
                count = forgery_types[forgery_type]
                percentage = count / fake_count * 100
                print(f"   {forgery_type}: {count} images ({percentage:.1f}%)")
    
    # Analyse de l'équilibre global des classes
    print(f"\nANALYSE DE L'ÉQUILIBRE DES CLASSES")
    print("="*40)
    
    total_authentic = (df_train['is_authentic'] == 1).sum()
    total_fake = (df_train['is_authentic'] == 0).sum()
    balance_ratio = total_authentic / total_fake if total_fake > 0 else 0
    
    print(f"Ratio Authentique/Falsifié: {balance_ratio:.2f}")
    
    if balance_ratio > 1.5 or balance_ratio < 0.67:
        print("ATTENTION: Classes déséquilibrées détectées!")
        print("Recommandations:")
        print("   - Utiliser des métriques équilibrées (F1-score, AUC-ROC)")
        print("   - Considérer des techniques de rééchantillonnage")
        print("   - Ajuster les poids des classes dans le modèle")
    else:
        print("Classes relativement équilibrées")
    
    # Analyse de la distribution des fichiers
    print(f"\nSTATISTIQUES GÉNÉRALES")
    print("="*25)
    print(f"Types de documents uniques: {df_train['document_type'].nunique()}")
    print(f"Classes uniques: {df_train['class_name'].nunique()}")
    print(f"Total d'images: {len(df_train):,}")
    
    # Exemples de noms de fichiers pour comprendre la nomenclature
    print(f"\nExemples de nomenclature:")
    for doc_type in df_train['document_type'].unique()[:2]:
        examples = df_train[df_train['document_type'] == doc_type]['filename'].head(3).tolist()
        print(f"   {doc_type}: {examples}")
        
else:
    print("Pas de données pour l'analyse détaillée")